In [24]:
import QUANTAXIS as QA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from talib import *
import time
import datetime


In [25]:
'''
QA_code_list = QA.QA_fetch_stock_list_adv()
QA_code_list
QA_code = QA_code_list.code.tolist() 
QA_code
QA_qfq_data = QA.QA_fetch_stock_day_adv(QA_code,start,end).to_qfq() 
QA_qfq = QA_qfq_data.data
QA_qfq
'''

'\nQA_code_list = QA.QA_fetch_stock_list_adv()\nQA_code_list\nQA_code = QA_code_list.code.tolist() \nQA_code\nQA_qfq_data = QA.QA_fetch_stock_day_adv(QA_code,start,end).to_qfq() \nQA_qfq = QA_qfq_data.data\nQA_qfq\n'

In [26]:
def market_list():
    import QUANTAXIS as QA
    return list(QA.QA_fetch_stock_list_adv().code)

In [29]:
# 定义均线金叉、长期均线上扬的选股函数
def ma_select(code_list=market_list(), date=None, short=10, long=100):
#def ma_select(code_list= QA_code_list.code.tolist() , date=None, short=10, long=100):
    """
    :param code_list: 选股范围，默认全市场
    :param date: 选股日期，默认None为现在
    :param short: 短期均线
    :param long: 长期均线
    :return: 返回选后的股票列表
    """
    now = datetime.now().strftime('%Y-%m-%d')
    if date is None:
        date = now
    date = QA.QA_util_get_real_date(date)
    if date == now and datetime.now().strftime('%H:%M') < '09:30':
        date = QA.QA_util_get_last_day(date, n=1)
    long_days_ago = QA.QA_util_get_last_day(date=date, n=long)
    data = QA.QA_fetch_stock_day_adv(code=code_list, start=long_days_ago, end=date)
    data = data.to_qfq()
    ind_data = data.add_func(QA.QA_indicator_MA, short, long)
    # 选取最后两日的数据
    ind_1 = ind_data.xs(date, level=0)
    ind_2 = ind_data.xs(QA.QA_util_get_last_day(date, 1), level=0)
    # 最后两日都有数据的股票代码
    code = list(set(ind_1.index) & set(ind_2.index))
    # 将最后两日的数据切齐
    ind_1 = ind_1.reindex(code)
    ind_2 = ind_2.reindex(code)
    # 选取最后一天均线大于前一天均线的代码列表
    code = list(ind_1[ind_1['MA'+str(long)] > ind_2['MA'+str(long)]].index)
    ind_1 = ind_1.reindex(code)
    # 选取均线金叉的股票代码
    code = list(ind_1[ind_1['MA'+str(short)] > ind_1['MA'+str(long)]].index)
    return code
